# Reorientation lifetime of vectors

In [ ]:
from fishmol import trj, atoms, utils, funcs, style
from cage_data import cage1_info
import warnings
import itertools
import time, sys

## Read trajectory file

In [ ]:
%%time
cell = cage1_info.cell
traj = trj.Trajectory(timestep = 5, data = "/nobackup/rhtp48/data_ana/cage1-500K.xyz", index = ":", cell = cell)

## Vector reorientation dynamics (VRD) of water O-H bonds

In [ ]:
water_vrd = new_VRD(traj = traj, spec = [[14,],[15,16]], num = 500, sampling = 10, skip = 2)
results = water_vrd.calculate(plot = False, l = 1)

In [ ]:
results.c_t_mean[:,0]

In [ ]:
x_fit, y_fit, params = water_vrd.kww_func_fit(results.t, results.c_t_mean[:,0])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(results.t, results.c_t_mean[:,0])
ax.plot(x_fit, y_fit)
plt.show()

## VRD of trifluoroacetate ion C-C bond

In [ ]:
TFAs = cage1_info.TFAs

In [ ]:
spec = [*TFAs[0].values()]
vrd = funcs.VRD(traj = traj, spec = [[spec[-2],], [spec[-1],]], num = 2000, sampling = 5, skip = 2)
results = vrd.calculate(plot = False, l = 3)